<a href="https://colab.research.google.com/github/daniel-sjkdm/ConsumingAPIs/blob/master/ConsumingSpotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spotify API

To connect to the Spotify API you should create an app in the dashboard section of the developer's page:

https://developer.spotify.com/dashboard/applications

When the app is created a client id and client secret are generated that we need to use to authenticate and make http request to the API.

Steps:

1. Store the credentials in a file that you only have access (like a .env file)
2. Generate an OAuth token that matchs your needs (see permissions at [1])
3. Make https requests and send the auth token as an authorization header

In [113]:
!pip install python-dotenv -q

In [130]:
import os
import base64
import pandas as pd
import requests 
from pprint import pprint
from dotenv import load_dotenv
from google.colab import drive

In [19]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Client Credentials

I'll store the credentias at my Google Drive filesystem inside a .env file.

Before generating the auth token to make http calls, the client credentials must be encoded in base64 as "client_id:client_secret". Make sure to fist encode them as UTF-8 format.

In [28]:
load_dotenv(dotenv_path="/content/drive/My Drive/Colab Notebooks/ConsumingAPIs/ConsumingSpotify/.env")

CLIENTID=os.getenv("CLIENTID")
CLIENTSECRET=os.getenv("CLIENTSECRET")

In [26]:
def encodebase64(data):
  return base64.b64encode(data)

def decodebase64(data):
  return base64.b64decode(data)

In [33]:
B64CLIENT = encodebase64((CLIENTID + ":" + CLIENTSECRET).encode("UTF-8"))

## Generating the OAuth token

The OAuth token is generated via a post request using basic authentication, sending the base64 encoded version of the client credentials generated in the spotify developer's dashboard.

There are many auth methods and scopes available at [1], for this notebook I'll use the "Authorization Code" flow and the "user-read-email" to verify it's working as expected.

[1] [Spotify Auth Guide](https://developer.spotify.com/documentation/general/guides/authorization-guide/)

In [91]:
# The token has an expiration date of 3600 seconds, to avoid
# refreshing it in this notebook, it's generated a new one 
# each time a request is performed

def generate_auth_token(b64client):
  url = "https://accounts.spotify.com/api/token"
  headers = {
      "Authorization": f"Basic {b64client.decode()}"
  }
  payload = {
      "grant_type": "client_credentials",
      "scopes": "user-read-email"
  }
  response = requests.post(url, data=payload, headers=headers)
  return response.json()

In [149]:
token = generate_auth_token(B64CLIENT)
print("Expires in: ", token["expires_in"])
print("Scopes: ", token["scope"])
print("Type: ", token["token_type"])

Expires in:  3600
Scopes:  user-read-email
Type:  Bearer


## Making HTTP requests

There are many endpoints available to make calls to, some of them are:

+ albums (Base URL: https://api.spotify.com/v1)
  + /albums/{id}
  + /albums/{id}/tracks
  + /albums

+ artists
  + /artists
  + /artists/{id}
  + ...


The response is in json format and must be parsed in a different way for each kind of search.

[Full list endpoints](https://developer.spotify.com/documentation/web-api/reference/)

In [268]:
base_url = "https://api.spotify.com/v1"

def generate_auth_headers():
  return {
    "Authorization": f"Bearer {generate_auth_token(B64CLIENT)['access_token']}",
    "Content-Type": "application/json",
    "Accept": "application/json"
  }

In [269]:
# Search method to look for albums, artists, playlists, shows and episodes
def search(query, search_type="track", limit=20):
  allowed_types = ["album", "artist", "playlist", "track", "show", "episode"]
  if search_type not in allowed_types:
    raise Exception("Invalid search type")
  if limit < 1 or limit > 50:
    raise Exception("Invalid limit value 1 <= limit <= 50")
  url = base_url + "/search"
  response = requests.get(
      url,
      headers = generate_auth_headers(),
      params = {
          "q": query,
          "type": search_type,
          "limit": limit
      }
  )
  return response.json()

In [270]:
# Print tracks function (parse the response for tracks)

def print_tracks(tracks):
  for index, track in enumerate(tracks["tracks"]["items"]):
    track_id = track["id"]
    track_name = track["name"]
    track_duration = track["duration_ms"] * 0.001
    track_popularity = track["popularity"]
    track_number = track["track_number"]

    album_id = track["album"]["id"]
    album_name = track["album"]["name"]
    album_artists = [ (artist["id"], artist["name"]) for artist in track["album"]["artists"] ]

    print("\nTrack")
    print("Track id: ", track_id)
    print("Track name: ", track_name)
    print("Track duration: ", track_duration)
    print("Track popularity: ", track_popularity)
    print("\nAlbum")
    print("Album id: ", album_id)
    print("Album name: ", album_name)
    print("Album artists:")
    for i, artist in enumerate(album_artists):
      print("\n\tArtist ", i)
      print("\tId:", artist[0])
      print("\tName:", artist[1])
    print("-------------------------------")

In [272]:
items = search("Vivir para contarlo/ Haciendo Lo Nuestro", search_type="track", limit=1)

print_tracks(items)


Track
Track id:  6RWB3HBIIggX3eo5BrDZeC
Track name:  Vivir para Contarlo
Track duration:  346.913
Track popularity:  52

Album
Album id:  3MMyvg2MoM23KIuGCjo28u
Album name:  Vivir para Contarlo/ Haciendo Lo Nuestro
Album artists:

	Artist  0
	Id: 1DH9RJ0xBVje6gQmK3LWUY
	Name: Violadores Del Verso
-------------------------------


In [167]:
# Function to print albums
def print_albums(albums):
  for index, album in enumerate(albums["albums"]["items"]):

    album_id = album["id"]
    album_name = album["name"]
    album_release_date = album["release_date"]
    album_tracks = album["total_tracks"]
    album_artists = [ (artist["id"], artist["name"]) for artist in album["artists"] ]

    print("\nAlbum ", index)
    print("Id: ", album_id)
    print("Name: ", album_name)
    print("Tracks: ", album_tracks)
    print("Artists:")
    for i, artist in enumerate(album_artists):
      print("\n\tArtist ", i)
      print("\tId:", artist[0])
      print("\tName:", artist[1])
    print("-------------------------------")

In [285]:
albums = search("Iron Maiden", search_type="album", limit=3)
pprint(albums)
print_albums(albums)

{'albums': {'href': 'https://api.spotify.com/v1/search?query=Iron+Maiden&type=album&offset=0&limit=3',
            'items': [{'album_type': 'album',
                       'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6mdiAmATAx73kdxrNrnlao'},
                                    'href': 'https://api.spotify.com/v1/artists/6mdiAmATAx73kdxrNrnlao',
                                    'id': '6mdiAmATAx73kdxrNrnlao',
                                    'name': 'Iron Maiden',
                                    'type': 'artist',
                                    'uri': 'spotify:artist:6mdiAmATAx73kdxrNrnlao'}],
                       'available_markets': ['US'],
                       'external_urls': {'spotify': 'https://open.spotify.com/album/5nyyw7ThJdClJ0jPisOta3'},
                       'href': 'https://api.spotify.com/v1/albums/5nyyw7ThJdClJ0jPisOta3',
                       'id': '5nyyw7ThJdClJ0jPisOta3',
                       'images': [{'height': 64

In [274]:
# Function to print artists
def print_artists(artists):
  for index, artist in enumerate(artists["artists"]["items"]):
    artist_id = artist["id"]
    artist_followers = artist["followers"]["total"]
    artist_name = artist["name"]
    artist_popularity = artist["popularity"]
    artists_genres = artist["genres"]

    print(f"\nArtist {index}")
    print("Id: ", artist_id)
    print("Name: ", artist_name)
    print("Followers: ", artist_followers)
    print("Popularity: ", artist_popularity)
    print("Genres: ", artists_genres)

In [275]:
artists = search("Black Sabbath", search_type="artist", limit=2)

print_artists(artists)


Artist 0
Id:  5M52tdBnJaKSvOpJGz8mfZ
Name:  Black Sabbath
Followers:  4761569
Popularity:  76
Genres:  ['album rock', 'birmingham metal', 'classic rock', 'hard rock', 'metal', 'rock', 'stoner rock', 'uk doom metal']

Artist 1
Id:  3pwXydggoKOQQaavtatHIY
Name:  Black Sabbath Riot
Followers:  144
Popularity:  0
Genres:  []


In [283]:
# Get an artists by its id

def get_artist(artist_id):
  url = base_url + "/artists" + "/" + artist_id
  artist = requests.get(
      url, 
      headers = generate_auth_headers(),
  )
  print(artist.url)
  return artists


artist = get_artist("5M52tdBnJaKSvOpJGz8mfZ")

print_artists(artist)

https://api.spotify.com/v1/artists/5M52tdBnJaKSvOpJGz8mfZ

Artist 0
Id:  5M52tdBnJaKSvOpJGz8mfZ
Name:  Black Sabbath
Followers:  4761569
Popularity:  76
Genres:  ['album rock', 'birmingham metal', 'classic rock', 'hard rock', 'metal', 'rock', 'stoner rock', 'uk doom metal']

Artist 1
Id:  3pwXydggoKOQQaavtatHIY
Name:  Black Sabbath Riot
Followers:  144
Popularity:  0
Genres:  []


In [296]:
# Get an album by its id

def get_album(album_id):
  url = base_url + "/albums" + "/" + album_id
  album = requests.get(
      url, 
      headers = generate_auth_headers(),
  )
  return album.json()

album = get_album("5nyyw7ThJdClJ0jPisOta3")


title = album["name"]
genres = album["genres"]
popularity = album["popularity"]
release_date = album["release_date"]
total_tracks = album["total_tracks"]
artists = [ artist["name"] for artist in album["artists"] ]


print("Title: ", title)
print("Genres: ", genres)
print("Popularity: ", popularity)
print("Release date: ", release_date)
print("Total tracks: ", total_tracks)
print("Tracks:")
for track in album["tracks"]["items"]:
  print("Tile: ", track["name"])

Title:  Iron Maiden (2015 - Remaster)
Genres:  []
Popularity:  49
Release date:  1980-04-14
Total tracks:  8
Tracks:
Tile:  Prowler - 2015 Remaster
Tile:  Remember Tomorrow - 2015 Remaster
Tile:  Running Free - 2015 Remaster
Tile:  Phantom of the Opera - 2015 Remaster
Tile:  Transylvania - 2015 Remaster
Tile:  Strange World - 2015 Remaster
Tile:  Charlotte the Harlot - 2015 Remaster
Tile:  Iron Maiden - 2015 Remaster
